Nome: Victor Henrique Oliveira Macaiba

Email: vhom@cesar.school

Dataset: https://www.kaggle.com/datasets/muratkokludataset/pistachio-dataset

**Explicação do dataSet:**

Este dataset contém 28 caracteristicas de 2 tipos de pistaches diferentes. Nosso objetivo é realizar o treinamento do modelo para que o mesmo possa, através de determinadas caracteristicas, determinar qual tipo de pistache é.

1. Area
2. Perimeter
3. Major_Axis
4. Minor_Axis
5. Eccentricity
6. Eqdiasq
7. Solidity
8. Convex_Area
9. Extent
10. Aspect_Ratio
11. Roundness
12. Compactness

Shape Features (4 Features)
13. Shapefactor_1
14. Shapefactor_2
15. Shapefactor_3
16. Shapefactor_4

Color Features (12 Features)
17. Mean_RR
18. Mean_RG
19. Mean_RB
20. StdDev_RR
21. StdDev_RG
22. StdDev_RB
213. Skew_RR
24. Skew_RG
25. Skew_RB
26. Kurtosis_RR
27. Kurtosis_RG
28. Kurtosis_RB

29. Class


Bibliotecas utilizadas

In [146]:
%matplotlib inline
from IPython import display
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
from torch.autograd import Variable
import torch.nn as nn
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from torchvision import datasets, transforms
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import torch.nn as nn


Carregamento

In [147]:
pistache = pd.read_excel('https://github.com/VictorMacaiba/RNA-e-Deep-Learning---CESAR-School-2023.02/raw/main/Trabalho%20Final/Pistachio_Dataset/Pistachio_28_Features_Dataset/Pistachio_28_Features_Dataset.xlsx')

In [148]:
pistache.head()

,Area,Perimeter,Major_Axis,Minor_Axis,Eccentricity,Eqdiasq,Solidity,Convex_Area,Extent,Aspect_Ratio,...,StdDev_RR,StdDev_RG,StdDev_RB,Skew_RR,Skew_RG,Skew_RB,Kurtosis_RR,Kurtosis_RG,Kurtosis_RB,Class
0,63391,1568.405,390.3396,236.7461,0.7951,284.0984,0.8665,73160,0.6394,1.6488,...,17.7206,19.6024,21.1342,0.4581,0.6635,0.7591,2.9692,3.0576,2.9542,Kirmizi_Pistachio
1,68358,1942.187,410.8594,234.7525,0.8207,295.0188,0.8765,77991,0.6772,1.7502,...,26.7061,27.2112,25.1035,-0.3847,-0.2713,-0.2927,1.9807,2.1006,2.2152,Kirmizi_Pistachio
2,73589,1246.538,452.3630,220.5547,0.8731,306.0987,0.9172,80234,0.7127,2.0510,...,19.0129,20.0703,20.7006,-0.6014,-0.4500,0.2998,3.5420,3.6856,4.1012,Kirmizi_Pistachio
3,71106,1445.261,429.5291,216.0765,0.8643,300.8903,0.9589,74153,0.7028,1.9879,...,18.1773,18.7152,29.7883,-0.6943,-0.6278,-0.7798,2.8776,2.8748,2.8953,Kirmizi_Pistachio
4,80087,1251.524,469.3783,220.9344,0.8823,319.3273,0.9657,82929,0.7459,2.1245,...,23.4298,24.0878,23.1157,-0.9287,-0.8134,-0.4970,2.9915,2.8813,2.7362,Kirmizi_Pistachio


Tratamento para quando a classe for 'Kirmizi_Pistachio' a coluna tipo seja 0 e quando for 'Siit_Pistachio' seja 1.

In [149]:
pistache['tipo'] = np.where(pistache['Class'] == 'Kirmizi_Pistachio', 0, 1)

In [150]:
pistache['tipo']

0       0
1       0
2       0
3       0
4       0
       ..
2143    1
2144    1
2145    1
2146    1
2147    1
Name: tipo, Length: 2148, dtype: int64

In [151]:
pistacheT = pistache.drop(['Class'], axis=1)

Dividindo dataset em treino e teste

In [152]:
y = pistacheT['tipo']
#x = pistacheT.drop(['tipo'], axis=1).values

In [153]:
x = pistacheT['Kurtosis_RR'], pistacheT['StdDev_RR'], pistacheT['Skew_RR'], pistacheT['StdDev_RG'], pistacheT['Skew_RG'], pistacheT['Kurtosis_RG'], pistacheT['StdDev_RB'], pistacheT['Skew_RB'], pistacheT['Kurtosis_RB']#, pistacheT['Mean_RB'], pistacheT['Mean_RR'], pistacheT['Mean_RG']
x = pd.DataFrame(x)
x = x.T

In [154]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42, shuffle=True)

In [155]:
x_train

,Kurtosis_RR,StdDev_RR,Skew_RR,StdDev_RG,Skew_RG,Kurtosis_RG,StdDev_RB,Skew_RB,Kurtosis_RB
1436,4.2729,17.2444,-0.8847,16.7899,-0.8592,4.1289,14.0778,-0.3492,4.1676
1173,2.6295,27.1898,-0.8469,29.7259,-0.7178,2.4524,30.5436,-0.5916,2.2173
845,4.1709,20.6819,-1.1655,20.7985,-1.0467,3.8791,19.3268,-0.8329,3.7103
526,3.2149,20.0879,-0.9517,21.7255,-0.8007,2.9111,23.2332,-0.7240,3.0386
715,4.6826,21.6808,-1.4895,22.1918,-1.3498,4.2172,18.8748,-0.9973,3.8522
...,...,...,...,...,...,...,...,...,...
1638,3.4089,20.4556,-0.9604,20.4738,-0.7437,2.9411,18.5239,-0.1310,2.6859
1095,2.5664,21.7308,-0.2294,24.2249,0.0251,2.5919,28.3911,0.5039,2.6358
1130,4.1030,20.5878,-1.2335,20.9450,-1.0300,4.0707,19.6246,-0.5739,3.7449
1294,3.9919,20.3168,-1.1297,20.7078,-0.9430,3.5275,19.9043,-0.6065,3.2252


Transformando para utilização

In [156]:
x_trainTorch = torch.FloatTensor(x_train.values)
y_trainTorch = torch.LongTensor(y_train)

In [157]:
x_testTorch = torch.FloatTensor(x_test.values)
y_testTorch = torch.LongTensor(y_test.values)

In [158]:
X_tensor = x_trainTorch
Y_tensor = y_trainTorch

In [159]:
X_tensor.size()

torch.Size([1503, 9])

Normalização

In [160]:
X =  x_train
Xc = X - X.min(axis=0)
Xc /= Xc.max(axis=0)
Y =  y_train

Definição do modelo

In [161]:
class Modelo(nn.Module):
    def __init__(self):
        super(Modelo, self).__init__()
        self.dense = nn.Sequential(
            nn.Linear(9, 30),
            nn.ReLU(),
            nn.Linear(30, 30),
            nn.ReLU(),
            nn.Linear(30, 60),
            nn.ReLU(),
            nn.Linear(60, 60),
            nn.ReLU(),
            nn.Linear(60, 30),
            nn.ReLU(),
            nn.Linear(30,15),
            nn.ReLU(),
            nn.Linear(15,2),
        )

    def forward(self, x):
        x = self.dense(x)

        return x

model = Modelo()
print(model)

Modelo(
  (dense): Sequential(
    (0): Linear(in_features=9, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=30, bias=True)
    (3): ReLU()
    (4): Linear(in_features=30, out_features=60, bias=True)
    (5): ReLU()
    (6): Linear(in_features=60, out_features=60, bias=True)
    (7): ReLU()
    (8): Linear(in_features=60, out_features=30, bias=True)
    (9): ReLU()
    (10): Linear(in_features=30, out_features=15, bias=True)
    (11): ReLU()
    (12): Linear(in_features=15, out_features=2, bias=True)
  )
)


In [162]:
def predict(model, inputs):
    outputs = model(Variable(inputs))
    _, predicts = torch.max(outputs, 1)

    return predicts.data.numpy()

In [163]:
lr = 0.15

optimizer = torch.optim.SGD(model.parameters(), lr=lr)

criterion = nn.CrossEntropyLoss()

Treinamento e erro final:

In [164]:
batch_size = 200
nb_epoch = 15000

for i in range(nb_epoch):
    # predict da rede
    output = model(Variable(X_tensor))

    loss = criterion(output, Variable(Y_tensor))

    # zero, backpropagation gradient, atualiza para gradiente descendente
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print('Final loss:', loss.data)

Final loss: tensor(0.4029)


Acurácia de treino:

In [165]:
Y_pred = predict(model, x_trainTorch)
accuracy = (Y_pred == y_train).mean()
print('Training Accuracy: ', accuracy)

Training Accuracy:  0.7877578176979375


Matriz de Confusão do treino:

In [166]:
pd.crosstab(Y_pred, y_train)


tipo,0,1
row_0,,
0,650,109
1,210,534


Acurácia de teste:

In [167]:
Y_pred = predict(model, x_testTorch)
accuracy = (Y_pred == y_test).mean()
print('Training Accuracy: ', accuracy)

Training Accuracy:  0.7689922480620155


Matriz de Confusão do teste:

In [168]:
pd.crosstab(Y_pred, y_test)


tipo,0,1
row_0,,
0,273,50
1,99,223


**Explicação do processo:**

A escolha do dataset pistache foi realizada devido a próximidade com o dataset Iris, no qual foi trabalhado durante o curso. Com isso, foi estabeleci que usária apenas algumas features para alcançar o resultado acima dos 70%

Inicialmente foi escolhido o dataset com 16 features, mas após vários treinos e tentativas, foi observado que a acurácia não passava dos 55%. Com este fato, foi adotado o dataset de 28 features e dentro dele foram trabalhados algumas combinações de caracteristicas para que se atingisse uma acurácia acima dos 70%.

A dificuldade sentida foi principalmente por conta dos cruzamentos das várias caracteristicas semelhantes entre os dois tipos de pistaches. Após a definição das caracteriscas com base na maior diferenciação delas para cada tipo de pistache, foi desenhada a rede com ReLU e algumas camadas de neôronios para que o modelo podesse ser bem desenvolvido. Foi definido um batch_size de 200, lr de 0.15 e 15000 epócas para desenvolviemnto do modelo o que resultou uma acurácia de 78,7% na base de treino e 76,8% no teste, sendo eles dividos em 70 e 30% respectivamente.

O modelo apresentou mais dificuldade em aprender diferenciar o tipo 1 do tipo 0, apresentando muitos falsos positivos.

